In [111]:
from keras.engine import  Model
from keras.layers import Flatten, Dense, Input, Dropout
from keras_vggface.vggface import VGGFace
from keras_vggface import utils
from keras.optimizers import Adam
from keras.preprocessing import image
from keras import backend as K
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [132]:
K.clear_session()

In [133]:
vgg_base = VGGFace(include_top=False, input_shape=(224, 224, 3), pooling='max')

In [134]:
vgg_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)       0         
__________

In [135]:
# Add custom layers
last_layer = vgg_base.get_layer('global_max_pooling2d_1').output
X = Dropout(0.2)(last_layer)
# x = Flatten(name='flatten')(last_layer)
X = Dense(256, activation='relu', name='fc6')(X)
X = Dropout(0.2)(X)
X = Dense(128, activation='relu', name='fc7')(X)
X = Dropout(0.2)(X)
output = Dense(1, activation='linear')(X)

In [136]:
# this is the model we will train
model = Model(inputs=vgg_base.input, outputs=output)

In [137]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in vgg_base.layers:
    layer.trainable = False

In [138]:
# Make sure weights are not trainable
model.trainable_weights

[<tf.Variable 'fc6/kernel:0' shape=(512, 256) dtype=float32_ref>,
 <tf.Variable 'fc6/bias:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'fc7/kernel:0' shape=(256, 128) dtype=float32_ref>,
 <tf.Variable 'fc7/bias:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'dense_1/kernel:0' shape=(128, 1) dtype=float32_ref>,
 <tf.Variable 'dense_1/bias:0' shape=(1,) dtype=float32_ref>]

In [139]:
model.compile(optimizer=Adam(clipnorm=1.0),loss='mean_absolute_error', metrics=['mae'])

In [110]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)       0         
__________

In [84]:
BASE_DIR = 'C:\\Users\\Lenovo 500\\Desktop\\Projects\\LoveClassifier\\all_females'

In [114]:
# IMPORTANT !
# LISTDIR returns the files as indexed by the filessystem
# not necessarily in alphanumeric order, so need to sort them!
N_IMAGES = len(os.listdir(BASE_DIR))
IMAGE_SIZE = (224, 224)

X_train = np.zeros((N_IMAGES, IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
y_train = np.zeros(N_IMAGES)

for idx, _im in enumerate(sorted(os.listdir(BASE_DIR))):
    # Change the image path with yours.
    _img = image.load_img(os.path.join(BASE_DIR, _im), target_size=IMAGE_SIZE)
    _x = image.img_to_array(_img)
    _x = np.expand_dims(_x, axis=0)
    X_train[idx, :, :, :] = utils.preprocess_input(_x, version=1) / .255 

In [126]:
# Make sure images are ok
assert ~np.any(np.isnan(X_train))

In [146]:
# Load ratings
ratings = np.genfromtxt('./Projekt_SGE_Assessment_ErikK.txt')

In [147]:
# Standardize ratings
ratings_z = (ratings - np.mean(ratings)) / np.std(ratings)

In [151]:
history = model.fit(X_train, ratings, batch_size=64, epochs=5, validation_split=0.2)

Train on 454 samples, validate on 114 samples
Epoch 1/5
454/454 [==============================] - 607s - loss: 175.7659 - mean_absolute_error: 175.7659 - val_loss: 85.0006 - val_mean_absolute_error: 85.0006
Epoch 2/5
454/454 [==============================] - 597s - loss: 66.0431 - mean_absolute_error: 66.0431 - val_loss: 32.8149 - val_mean_absolute_error: 32.8149
Epoch 3/5
454/454 [==============================] - 596s - loss: 44.7694 - mean_absolute_error: 44.7694 - val_loss: 9.8013 - val_mean_absolute_error: 9.8013
Epoch 4/5
454/454 [==============================] - 578s - loss: 27.8944 - mean_absolute_error: 27.8944 - val_loss: 8.3531 - val_mean_absolute_error: 8.3531
Epoch 5/5
454/454 [==============================] - 568s - loss: 19.2702 - mean_absolute_error: 19.2702 - val_loss: 4.4954 - val_mean_absolute_error: 4.4954


In [152]:
history2 = model.fit(X_train, ratings, batch_size=64, epochs=5, validation_split=0.2)

Train on 454 samples, validate on 114 samples
Epoch 1/5
454/454 [==============================] - 593s - loss: 16.2091 - mean_absolute_error: 16.2091 - val_loss: 7.0996 - val_mean_absolute_error: 7.0996
Epoch 2/5
454/454 [==============================] - 664s - loss: 11.2102 - mean_absolute_error: 11.2102 - val_loss: 3.3607 - val_mean_absolute_error: 3.3607
Epoch 3/5
454/454 [==============================] - 553s - loss: 8.1728 - mean_absolute_error: 8.1728 - val_loss: 2.5899 - val_mean_absolute_error: 2.5899
Epoch 4/5
454/454 [==============================] - 539s - loss: 4.4586 - mean_absolute_error: 4.4586 - val_loss: 1.3857 - val_mean_absolute_error: 1.3857
Epoch 5/5
454/454 [==============================] - 548s - loss: 3.8061 - mean_absolute_error: 3.8061 - val_loss: 1.3797 - val_mean_absolute_error: 1.3797


In [153]:
model.save('love_classifierv0.h5')